In [1]:
import sys, os
import rasterio

import pandas as pd
import geopandas as gpd

import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.dataMisc as dMisc
import GOSTrocks.mapMisc as mapMisc
from GOSTrocks.misc import tPrint

In [2]:
# Local/input files
iso3 = 'KHM'
out_folder = "c:/WBG/Work/KHM_Energy/data"
wsf_file = os.path.join(out_folder, "WSF", "wsf.tif")
ghsl_file = os.path.join(out_folder, "GHSL", "ghsl.tif")
overture_buildings = os.path.join(out_folder, "overture", "overture_download_2024_03_29.csv")
overture_raster = os.path.join(out_folder, "overture", "overture_download_2024_03_29.tif")
overture_raster_points = os.path.join(out_folder, "overture", "overture_download_2024_03_29_points.tif")
ghs_smod = os.path.join(out_folder, "URBAN", "GHS_SMOD.tif")
ghs_ucbd = os.path.join(out_folder, "URBAN", "GHS_UCBD.gpkg")
ntl_folder = os.path.join(out_folder, "NTL", "VIIRS_KHM")
google_buildings = os.path.join(out_folder, "Google_Buildings", "GOB_cambodia.shp")
gep_folder = os.path.join(out_folder, "GEP")
gep_settlements = os.path.join(gep_folder, "final_clusters.shp")
gep_attributes  = os.path.join(gep_folder, "kh-2-0_0_0_0_1_0.csv")

for file in [wsf_file, ghsl_file, ghs_smod]:
    if not os.path.exists(os.path.dirname(file)):
        os.makedirs(os.path.dirname(file))

# get country extent from geopandas
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
country = world[world.iso_a3 == iso3]
country['geometry'] = country.buffer(0.1)

C:\Users\WB411133\AppData\Local\Temp\ipykernel_28028\2764803234.py:22: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world_filepath = gpd.datasets.get_path('naturalearth_lowres')
Cannot find header.dxf (GDAL_DATA is not defined)
C:\Users\WB411133\AppData\Local\Temp\ipykernel_28028\2764803234.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  country['geometry'] = country.buffer(0.1)
C:\wbg\Anaconda3\envs\urban_test\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [ ]:
# Extract GHS data from global composite
if not os.path.exists(ghs_smod):
    tPrint("Extracting GHS data")
    ghs_file = r"J:\Data\GLOBAL\GHSL\SMOD\GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif"
    ghs = rMisc.clipRaster(rasterio.open(ghs_file), country, ghs_smod, crop=False)

In [ ]:
# Extract GHS UCBD data from global composite
if not os.path.exists(ghs_ucbd):
    tPrint("Extracting GHS UCBD data")
    ghs_file = r"J:\Data\GLOBAL\GHSL\GHS_UCBD_R2019A\GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg"
    in_extents = gpd.read_file(ghs_file)
    sel_extents = in_extents.loc[in_extents.intersects(country.unary_union)]
    sel_extents.to_file(ghs_ucbd, driver="GPKG")

In [3]:
# Join GEP settlements to GEP attributes
gep_d = gpd.read_file(gep_settlements)
gep_a = pd.read_csv(gep_attributes)

In [4]:
gep_d.head()

,Country,Population,ElecPop,NightLight,Area,id,geometry
0,Cambodia,4.547698,0.0,0.0,0.009,78860,"POLYGON ((103.80005 10.98277, 103.80005 10.982..."
1,Cambodia,8.725122,0.0,0.0,0.009,78844,"POLYGON ((103.87671 10.98528, 103.87671 10.985..."
2,Cambodia,4.547698,0.0,0.0,0.009,78835,"POLYGON ((103.80755 10.98361, 103.80755 10.983..."
3,Cambodia,6.051354,0.0,0.0,0.009,78931,"POLYGON ((103.78588 10.97694, 103.78588 10.976..."
4,Cambodia,8.725122,0.0,0.0,0.009,78915,"POLYGON ((103.90588 10.98111, 103.90588 10.981..."


In [5]:
gep_a.head()

,id,Cat_1,Cat_2,Cat_3,AgriDemand,CommercialDemand,Country,CurrentHVLineDist,CurrentMVLineDist,EducationDemand,...,MinGridDist2030,ElectrificationOrder2030,MinimumOverall2030,MinimumOverallLCOE2030,MinimumOverallCode2030,InvestmentCost2030,ElecStatusIn2030,InvestmentCapita2030,FinalElecCode2030,NewCapacity2030
0,78860,0,0,0,0,0,Cambodia,22.389,3.820,0.0,...,0.0,0,SA_PV2030,0.385118,3.0,3004.5845,1,543.75930,3.0,0.672167
1,78844,0,0,0,0,0,Cambodia,20.008,6.194,0.0,...,0.0,0,SA_PV2030,0.394091,3.0,5898.8420,1,556.42810,3.0,1.319651
2,78835,0,0,0,0,0,Cambodia,21.967,4.054,0.0,...,0.0,0,SA_PV2030,0.385357,3.0,3006.4526,1,544.09735,3.0,0.672585
3,78931,0,0,0,0,0,Cambodia,23.660,3.410,0.0,...,0.0,0,SA_PV2030,0.385118,3.0,2718.5908,1,369.74734,3.0,0.608186
4,78915,0,0,0,0,0,Cambodia,20.502,4.271,0.0,...,0.0,0,SA_PV2030,0.401977,3.0,6016.8920,1,567.56360,3.0,1.346061


In [15]:
gep_columns = ['id',"Pop","Pop2030","NumPeoplePerHH","IsUrban","NightLights","CurrentMVLineDist","PlannedMVLineDist",
               "FinalElecCode2020","FinalElecCode2030","NewConnections2025",'NewConnections2030']

sel_gep_a = gep_a.loc[:,gep_columns].copy()
sel_gep_a['nConnections'] =  sel_gep_a.apply(lambda x: x['NewConnections2030']/x['NumPeoplePerHH'], axis=1)
sel_gep_a.head()

,id,Pop,Pop2030,NumPeoplePerHH,IsUrban,NightLights,CurrentMVLineDist,PlannedMVLineDist,FinalElecCode2020,FinalElecCode2030,NewConnections2025,NewConnections2030,nConnections
0,78860,4.547697,5.525578,5.3,0,0.0,3.820,3.820,99,3.0,5.216904,5.525578,1.042562
1,78844,8.725122,10.601265,5.3,0,0.0,6.194,6.194,99,3.0,10.009048,10.601265,2.000239
2,78835,4.547697,5.525578,5.3,0,0.0,4.054,4.054,99,3.0,5.216904,5.525578,1.042562
3,78931,6.051354,7.352563,5.3,0,0.0,3.410,3.410,99,3.0,6.941828,7.352563,1.387276
4,78915,8.725122,10.601265,5.3,0,0.0,4.271,4.271,99,3.0,10.009048,10.601265,2.000239
